In [3]:
import os
import json
import pandas as pd
from parsel import Selector

In [5]:
def extract_data_from_html(filename):

    with open(f'./pages/{filename}', 'r') as f:
        
        selector = Selector(f.read())
        data = selector.css("script#__NEXT_DATA__::text").get()
        
        if data:
            # Option 1: some properties are located in NEXT DATA cache
            data = json.loads(data)
            property_data = json.loads(data["props"]["pageProps"]["componentProps"]["gdpClientCache"])
            property_data = property_data[list(property_data)[0]]['property']
        else:
            # Option 2: other times it's in Apollo cache
            data = selector.css("script#hdpApolloPreloadedData::text").get()
            data = json.loads(json.loads(data)["apiCache"])
            property_data = next(
                v["property"] for k, v in data.items() if "ForSale" in k
            )
    
        return {
            'zpid': property_data.get('zpid'),
            'street_address': property_data.get('streetAddress'),
            'bedrooms': property_data.get('bedrooms'),
            'bathrooms': property_data.get('bathrooms'),
            'price': property_data.get('price'),
            'year_built': property_data.get('yearBuilt'),
            'living_area': property_data.get('livingArea'),
            'home_type': property_data.get('homeType'),
            'lot_size': property_data.get('lotSize'),
            'lot_area_value': property_data.get('lotAreaValue')
        }

In [19]:
pages = os.listdir('./pages')
data = [ extract_data_from_html(fn) for fn in pages if fn.endswith('.html') ]

IsADirectoryError: [Errno 21] Is a directory: './pages/.ipynb_checkpoints'

In [14]:
final = pd.DataFrame(data)
final

,zpid,street_address,bedrooms,bathrooms,price,year_built,living_area,home_type,lot_size,lot_area_value
0,59054539,10 Beach Rd,3,1.0,268300,1953,984,SINGLE_FAMILY,10890.0,0.250000
1,59048464,27 Banfield Ln,3,1.5,263400,1964,1100,SINGLE_FAMILY,10454.0,10454.000000
2,59054533,31 Beatrice Ave,2,1.0,228500,1956,984,SINGLE_FAMILY,10454.0,10454.000000
3,2063439878,60 Barry Cir #43,2,1.5,0,2017,1200,APARTMENT,NaN,NaN
4,59048571,3 Beatrice Ave,3,1.5,243900,1956,994,SINGLE_FAMILY,17859.0,0.409986
5,59054533,31 Beatrice Ave,2,1.0,228500,1956,984,SINGLE_FAMILY,10454.0,10454.000000
6,174104759,94 Brookline Ave,3,2.5,286700,1914,1736,SINGLE_FAMILY,4791.0,4791.000000


In [16]:
final.to_csv('final.csv', index=False)
